In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
print("\nGPU Status:")
import tensorflow as tf
tf.test.gpu_device_name() #檢查GPU是否啟動，若無啟動則會自動改由CPU執


GPU Status:


'/device:GPU:0'

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
import sklearn
import xgboost
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style='whitegrid',context='notebook')

In [15]:
print("Pandas verion is {}".format(pd.__version__))
print("numpy verion is {}".format(np.__version__))
print("xgboost verion is {}".format(xgboost.__version__))
print("Sklearn verion is {}".format(sklearn.__version__))

Pandas verion is 1.1.5
numpy verion is 1.19.5
xgboost verion is 0.90
Sklearn verion is 0.22.2.post1


In [ ]:
# 資料載入
df = pd.read_csv(r'drive/MyDrive/AI比賽專案/智慧農業/訓練資料/train_data.csv')
data = df.drop(['Unnamed: 0'],axis=1)
label = df.loc[:,['actuator01','actuator02','actuator03','actuator04',
                  'actuator05','actuator06','actuator07','actuator08','actuator09','actuator10','actuator11']]

# 特徵工程

In [ ]:
# 新增早上或晚上 早上0 晚上1
def cal_time(hour,minute):
  time = 10*int(hour)+int(minute)
  if time <= 12:
    time = 0
  else:
    time = 1
  return time  

list1 = []
for i in range(data.shape[0]):
  a = data.iloc[i,0]
  a = a.split(' ')
  b = a[1].split(':')
  time = cal_time(b[0][0],b[0][1])
  list1.append(time)
data['d.time'] = list1   

In [ ]:
# 處理d.log_time，轉成分鐘
import pandas as pd
list1 = []
for i in range(data.shape[0]):
  a = data.iloc[i,0]
  a = a.split(' ')
  time = a[1].split(':')
  hour = time[0]
  minute = time[1]
  time = 0
  if hour[0] == '0':
    time = int(hour[1])*60
  else:
    time = int(hour)*60
  if minute[0] == '0':
    time = time + int(minute[1])
  else:
    time = time + int(minute)
  list1.append(time)
data['d.minute'] = list1

In [ ]:
# 新增天數
def sum_day(month,day):
  month_list = ['01','02','03','04','05','06','07','08','09','10','11','12']
  count = 0
  for m in month_list:
      if m[0] == '0':
        if m == '01':
          count = count
        elif m == '02':
          count = count + 31
        elif m == '03':
          count = count + 29
        elif int(m[1])%2 == 0:
          count = count + 31
        else:
          count = count + 30
      else:
        if m == '10':
          count = count + 31
        elif m == '11':
          count = count +30
        else:
          count = count + 31
      if m == month:
        day = int(day) + count
        break
  return day             

list1 = []
for i in range(data.shape[0]):
  a = data.iloc[i,0]
  a = a.split(' ')
  b = a[0].split('-')
  day = sum_day(b[1],b[2])
  list1.append(day)
data['d.day'] = list1   

In [ ]:
# 新增一個actuator10注水次數欄位
data['actuator10_water_times'] = -1

# 計算一天內每個時段的累積注水次數
day = '07/31'
list1 = []
count = 0 # 今日的注水次數
for i in range(data.shape[0]):
  a = data.iloc[i,0]
  a =  a.split(' ')
  b = a[0].split('-')
  day2 = b[1]+'/'+b[2]
  if day == day2:
      list1.append(count)
      if label['actuator10'][i] == 1:
        count = count + 1
  else:
      day = day2
      count = 0 
      list1.append(count)
      if label['actuator10'][i] == 1:
        count = count + 1
data['actuator10_water_times'] = list1

In [ ]:
# 新增一個actuator11注水次數欄位
data['actuator11_water_times'] = -1

# 計算一天內每個時段的累積注水次數
day = '07/31'
list1 = []
count = 0 # 今日的注水次數
for i in range(5):
  a = data.iloc[i,0]
  a =  a.split(' ')
  b = a[0].split('-')
  day2 = b[1]+'/'+b[2]
  if day == day2:
      list1.append(count)
      if label['actuator11'][i] == 1:
        count = count + 1
  else:
      day = day2
      count = 0 
      list1.append(count)
      if label['actuator11'][i] == 1:
        count = count + 1
data['actuator11_water_times'] = list1

In [ ]:
data = data.drop(['d.log_time','actuator01','actuator02','actuator03','actuator04','actuator05','actuator06','actuator07','actuator08','actuator09','actuator10','actuator11'],axis=1)

In [ ]:
data.columns

Index(['d.temperature_A', 'd.temperature_B', 'd.humidity_A', 'd.humidity_B',
       'd.photometric', 'd.CO2', 'd.soil_temperature', 'd.soil_humidity',
       'd.soil_EC', 'd.soil_PH', 'd.outside_photometric',
       'd.outside_temperature', 'd.outside_humidity', 'd.wind_speed',
       'd.wind_direction', 'd.radiometric', 'd.rainfall_detect', 'd.rainfall',
       'd.time', 'd.minute', 'd.day', 'actuator10_water_times',
       'actuator11_water_times'],
      dtype='object')

# 針對執行器

In [ ]:
# actuator10,actuator11的資料預處理
def get_data(n):
  temp1 = y_train[n]
  temp1 = np.asarray(temp1)
  label_0 = []
  label_1 = []
  data_0 = []
  data_1 = []
  for i in range(temp1.shape[0]):
    temp = temp1[i]
    if temp == 0:
      t = list(X_train.iloc[i,:])
      data_0.append(t)
      label_0.append([0])
    elif temp == 1:
      t = list(X_train.iloc[i,:])
      data_1.append(t)
      label_1.append([1])
  data_0 = np.asarray(data_0)
  label_0 = np.asarray(label_0)
  data_1 = np.asarray(data_1)
  label_1 = np.asarray(label_1)
  return data_0,label_0,data_1,label_1

def train_actuator10(data_0,label_0,data_1,label_1,model):
    count_old = 0
    count_new = 0
    index = 0
    for i in range(1,25):
      count_new = i*792
      train_data = np.vstack((data_0[count_old:count_new],data_1))
      train_data = pd.DataFrame(train_data,columns=X_train.columns)
      train_label = np.vstack((label_0[count_old:count_new],label_1))
      train_label = train_label.ravel()
      model.fit(train_data,train_label)
      if i == 1:
        count_old = count_new
      else:
        if index == 4:
          count_old = count_new
          index = 0
        else:
          index = index + 1
    return model
def train_actuator11(data_0,label_0,data_1,label_1,model):
    count_old = 0
    count_new = 0
    index = 0
    for i in range(1,40):
      count_new = i*488
      train_data = np.vstack((data_0[count_old:count_new],data_1))
      train_data = pd.DataFrame(train_data,columns=X_train.columns)
      train_label = np.vstack((label_0[count_old:count_new],label_1))
      train_label = train_label.ravel()
      model.fit(train_data,train_label)
      if i == 1:
        count_old = count_new
      else:
        if index == 8:
          count_old = count_new
          index = 0
        else:
          index = index + 1
    return model

# 建立模型

In [ ]:
def get_models():  # RandomForestClassifier(n_estimators=50,max_features='sqrt')
  models = dict()
  models['actuator01'] = XGBClassifier(n_estimators=500,
                                        learning_rate= 0.5,
                                        max_depth=7,
                                        min_child_weight=1,
                                        scale_pos_weight=1)
  models['actuator02'] = RandomForestClassifier(n_estimators=50,max_features=0.3)
  models['actuator03'] = RandomForestClassifier(n_estimators=50,max_features=0.3)
  models['actuator04'] = RandomForestClassifier(n_estimators=50,max_features=0.3)
  models['actuator05'] = RandomForestClassifier(n_estimators=200,max_features=0.4)
  models['actuator06'] = RandomForestClassifier(n_estimators=150,max_features=0.4)
  models['actuator07'] = RandomForestClassifier(n_estimators=150,max_features=0.5)
  models['actuator08'] = RandomForestClassifier(n_estimators=150,max_features=0.4)
  models['actuator09'] = XGBClassifier(n_estimators=500,
                                        learning_rate= 0.5,
                                        max_depth=6,
                                        min_child_weight=1,
                                        scale_pos_weight=10)
  models['actuator10'] = XGBClassifier(n_estimators=500,
                                        learning_rate= 0.5,
                                        max_depth=6,
                                        min_child_weight=1,
                                        scale_pos_weight=100)
  models['actuator11'] = XGBClassifier(n_estimators=500,
                                        learning_rate= 0.5,
                                        max_depth=6,
                                        min_child_weight=1,
                                        scale_pos_weight=100)
  return models

In [ ]:
# 模型訓練
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(data, 
                                                    label, 
                                                    test_size=0.3, 
                                                    random_state=1)
models = get_models()
list1 = ['actuator01','actuator02','actuator03','actuator04','actuator05','actuator06','actuator07','actuator08','actuator09','actuator10','actuator11']
# list1 = ['actuator01']
for i in list1:
  models[i].fit(X_train,y_train[i])
  y_predict = models[i].predict(X_test)
  print(i,' 分類準確率 : ',accuracy_score(y_test[i],y_predict))
  print(classification_report(y_test[i],y_predict,target_names=['0','1']))
  # plt.figure()
  # confusion_matrix = pd.crosstab(y_test[i], y_predict, rownames=['Actual'], colnames=['Predicted'])
  # sns.heatmap(confusion_matrix, annot=True)
  print('-------------------')

actuator01  分類準確率 :  0.9987131492746841
              precision    recall  f1-score   support

           0       1.00      0.94      0.97       171
           1       1.00      1.00      1.00      8377

    accuracy                           1.00      8548
   macro avg       1.00      0.97      0.98      8548
weighted avg       1.00      1.00      1.00      8548

-------------------
actuator02  分類準確率 :  0.9995320542817033
              precision    recall  f1-score   support

           0       1.00      0.95      0.97        80
           1       1.00      1.00      1.00      8468

    accuracy                           1.00      8548
   macro avg       1.00      0.97      0.99      8548
weighted avg       1.00      1.00      1.00      8548

-------------------
actuator03  分類準確率 :  0.9996490407112775
              precision    recall  f1-score   support

           0       1.00      0.96      0.98        79
           1       1.00      1.00      1.00      8469

    accuracy          

#測試

In [ ]:
# 測試模型準確率

# 資料預處理
print(y_test.shape)
test_label = []
for i in range(y_test.shape[0]):
  temp = y_test.iloc[i,:]
  test_label.append(list(temp))
print(len(test_label))

# 測試
list3 = ['actuator01','actuator02','actuator03','actuator04','actuator05','actuator06','actuator07','actuator08','actuator09','actuator10','actuator11']
count = 0 #總答對
temp2 = []
temp3 = []

# X_test.shape[0]
for i in range(200):
  temp = []
  print('回合 : ',(i+1))
  for n in list3: # 11
    data = X_test.iloc[i,:]
    data = np.array(data)
    data = data[np.newaxis,:]
    '''
    if n == 'actuator10' or n == 'actuator11':
      data = pd.DataFrame(data,columns=X_test.columns)
    '''
    r = models[n].predict(data)
    temp.append(r[0])
    print('>',end='')
  print()
  if temp == test_label[i]:
    count = count+1
  else:
    temp2.append(temp)
    temp3.append(test_label[i])
print(count)
print(count / 200)


(8548, 11)
8548
回合 :  1
>>>>>>>>>>>
回合 :  2
>>>>>>>>>>>
回合 :  3
>>>>>>>>>>>
回合 :  4
>>>>>>>>>>>
回合 :  5
>>>>>>>>>>>
回合 :  6
>>>>>>>>>>>
回合 :  7
>>>>>>>>>>>
回合 :  8
>>>>>>>>>>>
回合 :  9
>>>>>>>>>>>
回合 :  10
>>>>>>>>>>>
回合 :  11
>>>>>>>>>>>
回合 :  12
>>>>>>>>>>>
回合 :  13
>>>>>>>>>>>
回合 :  14
>>>>>>>>>>>
回合 :  15
>>>>>>>>>>>
回合 :  16
>>>>>>>>>>>
回合 :  17
>>>>>>>>>>>
回合 :  18
>>>>>>>>>>>
回合 :  19
>>>>>>>>>>>
回合 :  20
>>>>>>>>>>>
回合 :  21
>>>>>>>>>>>
回合 :  22
>>>>>>>>>>>
回合 :  23
>>>>>>>>>>>
回合 :  24
>>>>>>>>>>>
回合 :  25
>>>>>>>>>>>
回合 :  26
>>>>>>>>>>>
回合 :  27
>>>>>>>>>>>
回合 :  28
>>>>>>>>>>>
回合 :  29
>>>>>>>>>>>
回合 :  30
>>>>>>>>>>>
回合 :  31
>>>>>>>>>>>
回合 :  32
>>>>>>>>>>>
回合 :  33
>>>>>>>>>>>
回合 :  34
>>>>>>>>>>>
回合 :  35
>>>>>>>>>>>
回合 :  36
>>>>>>>>>>>
回合 :  37
>>>>>>>>>>>
回合 :  38
>>>>>>>>>>>
回合 :  39
>>>>>>>>>>>
回合 :  40
>>>>>>>>>>>
回合 :  41
>>>>>>>>>>>
回合 :  42
>>>>>>>>>>>
回合 :  43
>>>>>>>>>>>
回合 :  44
>>>>>>>>>>>
回合 :  45
>>>>>>>>>>>
回合 :  46
>>>>>>>>>>>
回合 :  47
>>>>>>>>>>>
回合 :  

#處理上傳資料

In [ ]:
# 需要先修改 test_data.csv，因為我有新增自己的特徵
test_data = pd.read_csv(r'drive/MyDrive/AI比賽專案/智慧農業/訓練資料/test_data.csv')
test_data = test_data.drop(['Unnamed: 0'],axis=1)

In [ ]:
# 有一點要注意訓練和預測資料的index順序要一樣，因此添加feature的順序要相同
'''
訓練時的
Index(['d.temperature_A', 'd.temperature_B', 'd.humidity_A', 'd.humidity_B',
       'd.photometric', 'd.CO2', 'd.soil_temperature', 'd.soil_humidity',
       'd.soil_EC', 'd.soil_PH', 'd.outside_photometric',
       'd.outside_temperature', 'd.outside_humidity', 'd.wind_speed',
       'd.wind_direction', 'd.radiometric', 'd.rainfall_detect', 'd.rainfall',
       'd.time', 'd.minute', 'd.day', 'actuator10_water_times',
       'actuator11_water_times'],
      dtype='object')
'''      

"\n訓練時的\nIndex(['d.temperature_A', 'd.temperature_B', 'd.humidity_A', 'd.humidity_B',\n       'd.photometric', 'd.CO2', 'd.soil_temperature', 'd.soil_humidity',\n       'd.soil_EC', 'd.soil_PH', 'd.outside_photometric',\n       'd.outside_temperature', 'd.outside_humidity', 'd.wind_speed',\n       'd.wind_direction', 'd.radiometric', 'd.rainfall_detect', 'd.rainfall',\n       'd.time', 'd.minute', 'd.day', 'actuator10_water_times',\n       'actuator11_water_times'],\n      dtype='object')\n"

In [ ]:
# 新增早上或晚上 早上0 晚上1
def cal_time(hour,minute):
  time = 10*int(hour)+int(minute)
  if time <= 12:
    time = 0
  else:
    time = 1
  return time  

list1 = []
for i in range(test_data.shape[0]):
  a = test_data.iloc[i,0]
  a = a.split(' ')
  b = a[1].split(':')
  time = cal_time(b[0][0],b[0][1])
  list1.append(time)
test_data['d.time'] = list1   

In [ ]:
# 處理d.log_time，轉成分鐘
import pandas as pd
list1 = []
for i in range(test_data.shape[0]):
  a = test_data.iloc[i,0]
  a = a.split(' ')
  time = a[1].split(':')
  hour = time[0]
  minute = time[1]
  time = 0
  if hour[0] == '0':
    time = int(hour[1])*60
  else:
    time = int(hour)*60
  if minute[0] == '0':
    time = time + int(minute[1])
  else:
    time = time + int(minute)
  list1.append(time)
test_data['d.minute'] = list1

In [ ]:
# 新增天數
def sum_day(month,day):
  month_list = ['01','02','03','04','05','06','07','08','09','10','11','12']
  count = 0
  for m in month_list:
      if m[0] == '0':
        if m == '01':
          count = count
        elif m == '02':
          count = count + 31
        elif m == '03':
          count = count + 29
        elif int(m[1])%2 == 0:
          count = count + 31
        else:
          count = count + 30
      else:
        if m == '10':
          count = count + 31
        elif m == '11':
          count = count +30
        else:
          count = count + 31
      if m == month:
        day = int(day) + count
        break
  return day             

list1 = []
for i in range(test_data.shape[0]):
  a = test_data.iloc[i,0]
  a = a.split(' ')
  b = a[0].split('-')
  day = sum_day(b[1],b[2])
  list1.append(day)
test_data['d.day'] = list1 

In [ ]:
test_data['actuator10_water_times'] = 0
test_data['actuator11_water_times'] = 0

In [ ]:
# 這裡有個問題，注水是一據預測結果計算的，所以必須一邊預測一邊更新資料
# 但是d.log_time會被拿掉，所以要用d.day判斷是不是同一天

In [ ]:
test_data = test_data.drop(['d.log_time'],axis=1)
test_data.columns == data.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [ ]:
data = test_data.iloc[0,:]
data = np.array(data)
data = data[np.newaxis,:]
data = pd.DataFrame(data)
data.columns = test_data.columns
print(data.columns)
a = data['d.day']
print(a == 217.0)
print(data.columns == test_data.columns)

In [ ]:
a = test_data['d.day'][0]
print(type(a))

<class 'numpy.int64'>


# 提交預測

In [ ]:
# 提交預測
submission = pd.read_csv(r'drive/MyDrive/AI比賽專案/智慧農業/結果/submission.csv')
submission = submission.drop(['Unnamed: 0'],axis=1)
name = ['actuator01','actuator02','actuator03','actuator04','actuator05','actuator06','actuator07','actuator08','actuator09','actuator10','actuator11']
actuator10_count = 0 # 計算注水次數
actuator11_count = 0 # 計算注水次數
old_day = test_data['d.day'][0] 

for i in range(test_data.shape[0]):
  data = test_data.iloc[i,:] # 讀入單筆資料
  data = np.array(data)
  data = data[np.newaxis,:]
  data = pd.DataFrame(data)
  data.columns = test_data.columns
  new_day = test_data['d.day'][i]
  
  if new_day != old_day: # 當變成不同天時
    actuator10_count = 0
    actuator11_count = 0
    old_day = new_day

  # 計算預測結果
  r = [] # 儲存結果
  print('回合 : ',i)
  for n in name:
    print('>',end='')
    pre = models[n].predict(data)
    r.append(pre)
    if n == 'actuator10' and pre == 1:
        actuator10_count = actuator10_count + 1
    if n == 'actuator11' and pre == 1:
        actuator11_count = actuator11_count + 1     
  
  if i != test_data.shape[0]-1: # 如果不是最後一筆
      next_line = i+1
      test_data.loc[next_line,'actuator10_water_times'] = actuator10_count # 讓下一筆資料可以知道今天到底注水多少
      test_data.loc[next_line,'actuator11_water_times'] = actuator11_count 
  print()
  submission.iloc[i,:] = r
submission.to_csv('drive/MyDrive/AI比賽專案/智慧農業/結果/submission_3月7號.csv')
test_data.to_csv('drive/MyDrive/AI比賽專案/智慧農業/結果/test_data_3月7號.csv')  
print('over')

In [ ]:
print(submission['actuator10'].value_counts())

0.0    12088
1.0      370
Name: actuator10, dtype: int64


In [ ]:
# 提交預測 舊版
submission = pd.read_csv(r'drive/MyDrive/AI比賽專案/智慧農業/結果/submission.csv')
submission = submission.drop(['Unnamed: 0'],axis=1)
# submission_1['actuator01'][0] = 100
# submission_1.to_csv('drive/MyDrive/AI比賽專案/智慧農業/結果/submission_1.csv')

for i in range(test_data.shape[0]):
  data = test_data.iloc[i,:]
  time = data[0] # 時間
  data = np.array(data)
  data = data[np.newaxis,:]
  temp = []
  print('回合 : ',i)
  if time == 0:
    test_data['actuator09_water_times'][i] = 0
    test_data['actuator09_water_times'][i:] = 0
    test_data['actuator10_water_times'][i] = 0
    test_data['actuator10_water_times'][i:] = 0
    test_data['actuator11_water_times'][i] = 0
    test_data['actuator11_water_times'][i:] = 0
  for n in list3:
    r = models[n].predict(data)
    temp.append(r)
    if n == 'actuator09' and r == 1 and i!=test_data.shape[0] -1:
      test_data['actuator09_water_times'][i+1] = test_data['actuator09_water_times'][i] + 1
      test_data['actuator09_water_times'][(i+1):] = test_data['actuator09_water_times'][i] + 1
    elif n == 'actuator10' and r == 1 and i!=test_data.shape[0] -1:
      test_data['actuator10_water_times'][i+1] = test_data['actuator10_water_times'][i] + 1
      test_data['actuator10_water_times'][(i+1):] = test_data['actuator10_water_times'][i] + 1
    elif n == 'actuator11' and r == 1 and i!=test_data.shape[0] -1:
      test_data['actuator11_water_times'][i+1] = test_data['actuator11_water_times'][i] + 1
      test_data['actuator11_water_times'][(i+1):] = test_data['actuator11_water_times'][i] + 1
    print('>',end='')
  print()
  submission.iloc[i,:] = temp
submission.to_csv('drive/MyDrive/AI比賽專案/智慧農業/結果/submission_3/7.csv')
print('over')
